# Actividad Integradora

Diego Antonio García Padilla (A01710777)

In [36]:
# Leer expresión regular (se asume que es correcta)

regex_input = input("Ingrese la expresión regular: ").lower()

print("La expresión regular es: ", regex_input)

La expresión regular es:  (a|b)*abb


In [37]:
# Parsear expresión regular
# Ejemplo: a*b(a|b) -> a*.b.(a|b)

def format_regex(regex_input):
    regex = ""
    
    # Agregar punto en concatenación
    for i in range(len(regex_input)):
        if i < len(regex_input) - 1:
            if regex_input[i].isalpha() and regex_input[i + 1].isalpha():
                regex += regex_input[i] + "."
            elif regex_input[i + 1] == "(":
                regex += regex_input[i] + "."
            elif regex_input[i] == ")" and regex_input[i + 1].isalpha():
                regex += regex_input[i] + "."
            else:
                regex += regex_input[i]
        else:
            regex += regex_input[i]
            
    # Tranformar cerradura positiva en cerradura de Kleene (a+ -> aa*)
    for i in range(len(regex)):
        if regex[i] == "+":
            l1 = regex[:i]
            l2 = regex[i + 1:]
            regex = l1 + l1[-1] + "*" + l2
        
    # Agregar punto entre operadores de repetición y sus operandos
    for i in range(len(regex)):
        if i < len(regex) - 1:
            if regex[i] == "*" and regex[i + 1].isalpha():
                regex = regex[:i + 1] + "." + regex[i + 1:]
            elif regex[i] == "*" and regex[i + 1] == "(":
                regex = regex[:i + 1] + "." + regex[i + 1:]
    
    # Retornar expresión regular formateada
    return regex

# Formatear expresión regular dos veces para asegurar que no queden puntos juntos
regex = format_regex(regex_input)

        
print("RegEx: ", regex)

RegEx:  (a|b)*.a.b.b


In [38]:
# Obtener alfabeto
# Ejemplo: a.*b.(a|b) -> {a, b}

alfabeto = set()

for i in range(len(regex)):
    if regex[i].isalpha():
        alfabeto.add(regex[i])

print("Alfabeto: ", alfabeto)

Alfabeto:  {'b', 'a'}


In [39]:
# Parsear expresión regular a posfijo

def regex_to_postfix(regex):
    postfix = ""
    stack = []
    
    for i in range(len(regex)):
        if regex[i].isalpha():
            postfix += regex[i]
        elif regex[i] == "(":
            stack.append(regex[i])
        elif regex[i] == ")":
            while stack[-1] != "(":
                postfix += stack.pop()
            stack.pop()
        else:
            while len(stack) > 0 and stack[-1] != "(":
                postfix += stack.pop()
            stack.append(regex[i])
    
    while len(stack) > 0:
        postfix += stack.pop()
    
    return postfix

postfix = regex_to_postfix(regex)

print("Posfijo: ", postfix)

Posfijo:  ab|*a.b.b.


In [40]:
# Generar autómata finito no determinista

expresiones = []
transiciones = {}

estados = -1
estado_inicial = 0
estado_final = 0

for char in postfix:
    # Si es un símbolo del alfabeto, generar un autómata finito no determinista con un estado inicial,
    # un estado final y una transición con el símbolo.
    if char in alfabeto:
        # Crear estados
        estados += 1
        estado_inicial = estados
        estados += 1
        estado_final = estados
        
        # Agregar estados a expresiones
        expresiones.append([estado_inicial, estado_final])
        
        # Agregar transición vacía de estado inicial a estado final
        transiciones[estado_inicial] = []
        
        # Agregar transición de estado inicial a estado final con el símbolo
        transiciones[estado_inicial].append((estado_final, char))
    
    # Si es un operador de repetición
    if char == "*":
        # Tomar la última expresión generada (para repetición es necesario tener una expresión)
        [e1, e2] = expresiones.pop()
        
        # Crear estados
        estados += 1
        estado_inicial = estados
        estados += 1
        estado_final = estados
        
        # Agregar estados a expresiones
        expresiones.append([estado_inicial, estado_final])
        
        # Agregar transiciones vacías
        transiciones[estado_inicial] = []
        transiciones[e2] = []
        
        # Agregar transiciones
        transiciones[estado_inicial].append((e1, "#"))
        transiciones[e2].append((estado_final, "#"))
        transiciones[estado_inicial].append((estado_final, "#"))
        transiciones[e2].append((e1, "#"))
    
    # Si es un operador de concatenación, unir los dos últimos autómatas finitos no deterministas generados
    if char == ".":
        # Tomar las dos últimas expresiones generadas (para concatenación es necesario tener dos expresiones)
        [e1, e2] = expresiones.pop() # 2do
        [e3, e4] = expresiones.pop() # 1er
        
        # Agregar transición vacia
        transiciones[e4] = []
        
        # Agregar transiciones
        transiciones[e4].append((e1, "#"))
        
        # Agregar estados a expresiones
        expresiones.append([e3, e2])
        
        # Actualizar estado inicial y final
        estado_inicial = e3
        estado_final = e2
    
    # Si es un operador de elección
    if char == "|":
        # Tomar las dos últimas expresiones generadas (para elección es necesario tener dos expresiones)
        [e1, e2] = expresiones.pop()
        [e3, e4] = expresiones.pop()
        
        # Crear estados
        estados += 1
        estado_inicial = estados
        estados += 1
        estado_final = estados
        
        # Agregar estados a expresiones
        expresiones.append([estado_inicial, estado_final])
        
        # Agregar transiciones vacías
        transiciones[estado_inicial] = []
        transiciones[e2] = []
        transiciones[e4] = []
        
        # Agregar transiciones
        transiciones[estado_inicial].append((e1, "#"))
        transiciones[estado_inicial].append((e3, "#"))
        transiciones[e2].append((estado_final, "#"))
        transiciones[e4].append((estado_final, "#"))
        
# Imprimir resultado
print("Estado inicial: ", estado_inicial)
print("Estado final: ", estado_final)

for transicion in transiciones:
    print("Transición: ", transicion, " -> ", transiciones[transicion])

Estado inicial:  6
Estado final:  13
Transición:  0  ->  [(1, 'a')]
Transición:  2  ->  [(3, 'b')]
Transición:  4  ->  [(2, '#'), (0, '#')]
Transición:  3  ->  [(5, '#')]
Transición:  1  ->  [(5, '#')]
Transición:  6  ->  [(4, '#'), (7, '#')]
Transición:  5  ->  [(7, '#'), (4, '#')]
Transición:  8  ->  [(9, 'a')]
Transición:  7  ->  [(8, '#')]
Transición:  10  ->  [(11, 'b')]
Transición:  9  ->  [(10, '#')]
Transición:  12  ->  [(13, 'b')]
Transición:  11  ->  [(12, '#')]
